# Ollama workshop notebook for IDX

## Setup instructions

ollama has been added in your IDX configuration 


First of all - we will need need to make sure we have the correct model downloaded

(you can browse models on the official [ollama site](https://ollama.com/search))


select a terminal - this will be serving ollama


```bash
ollama serve
```

now open a new terminal by clicking on the "+" icon

```bash
ollama pull gemma:2b
```

You can run the model by typing, you can test inference locally this way

```bash
ollama run gemma:2b
```

you can stop running a model for inference by typing

```bash
/exit
```

If you switch back to the terminal where ollama serve is running - you can see the API interactions/logs


Now, make sure the kernel for this notebook is your lab_env virtual environment




In [ ]:
# Install requirements

!pip install -r requirements.txt

In [ ]:
# Import libraries

import logging
from llama_index.core import SimpleDirectoryReader, Settings, VectorStoreIndex
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

In [4]:
# Define a simple logging - so we can see some of the details printed out


def set_logging():
    # Create a logger instance
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)  # Set the minimum logging level for the logger

    # Create a stream handler that outputs to the console
    stream_handler = logging.StreamHandler()
    stream_handler.setLevel(logging.INFO)  # Set the handler's level to INFO or DEBUG

    # Create a formatter to customize log output (optional, but recommended)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    stream_handler.setFormatter(formatter)

    # Add the stream handler to the logger
    logger.addHandler(stream_handler)



    #set up the global logger to have the same level and handler
    logging.basicConfig(level=logging.INFO, handlers=[stream_handler])

set_logging()

In [5]:
# 1. Load your data
logging.info("Loading documents...")
documents = SimpleDirectoryReader(input_dir="ollama-documents/").load_data()
logging.info(f"Loaded {len(documents)} documents.")

2025-01-02 13:36:24,737 - root - INFO - Loading documents...
2025-01-02 13:36:25,298 - root - INFO - Loaded 5 documents.


In [6]:
# 2. Setup Ollama LLM
# we want the embeddings to synthesize document info!
logging.info("Setting up Ollama LLM...")
Settings.llm = Ollama(model="gemma:2b", request_timeout=120.0)
logging.info("Setting up Ollama Embedding...")

ollama_embedding = OllamaEmbedding(
    model_name="gemma:2b",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0, "request_timeout": 120},
)


2025-01-02 13:36:36,542 - root - INFO - Setting up Ollama LLM...
2025-01-02 13:36:36,545 - root - INFO - Setting up Ollama Embedding...


In [7]:
# 3. Create the index

# This can take some time
logging.info("Creating index...")
index = VectorStoreIndex.from_documents(documents, embed_model=ollama_embedding)
logging.info("Setting up index completed...")

2025-01-02 13:36:40,740 - root - INFO - Creating index...
2025-01-02 13:36:51,784 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
2025-01-02 13:37:09,139 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
2025-01-02 13:37:30,150 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
2025-01-02 13:37:49,756 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
2025-01-02 13:38:08,871 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
2025-01-02 13:38:08,880 - root - INFO - Setting up index completed...


In [8]:
# 4. Query the index

#This will take more than a minute as well on IDX
logging.info("Querying index...")
query_engine = index.as_query_engine()

sitrep = query_engine.query("Scarecrow?")
print(sitrep)

2025-01-02 13:39:07,631 - root - INFO - Querying index...
2025-01-02 13:39:08,285 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
2025-01-02 13:40:17,490 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


The context does not provide information about Scarecrow, so I cannot answer this query from the provided context.
